In [ ]:
import sys
sys.path.append('/pycodesuggest/github-scraper')
sys.path.append('/pycodesuggest/lm')

import astwalker
import evaluation
import prebatcher
import pyreader
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime

from Trainer import Trainer
from batcher import QueuedSequenceBatcher, PreBatched
from hooks import SpeedHook, LossHook, SaveModelHook
from hooks import PerplexityHook

from utils import save_model, load_model, load_variables
from utils import get_file_list, copy_temp_files, create_model

flags = tf.flags
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string("data_path", "/data_normalised", "data path")
flags.DEFINE_boolean("train", False, "train the model")
flags.DEFINE_boolean("test", False, "test the model")
flags.DEFINE_boolean("preprocess", False, "Proprocess data")
flags.DEFINE_boolean("prebatch", False, "Pre-batch and split the data")
flags.DEFINE_boolean("vocab", False, "Generate vocab")
flags.DEFINE_string("list_file", "train_files.txt", "Name of the list file found in data_path")
flags.DEFINE_string("vocab_file", "mapping.map", "Name of the vocab file in data_path")
flags.DEFINE_string("output_file", "/data_normalised/out/all_train_data.dat", "Name of the output file")
flags.DEFINE_boolean("debug", False, "Use debug config")
flags.DEFINE_integer("seq_length", 100, "Sequence length")
flags.DEFINE_integer("batch_size", 100, "Batch size")
flags.DEFINE_integer("num_partitions", 10, "Data partitions")
flags.DEFINE_boolean("use_prebatched", True, "Use prebatched data")
flags.DEFINE_boolean("copy_temp", False, "Copy data to local temp directory")
flags.DEFINE_integer("oov_threshold", 0, "Out of vocabulary threshold")
flags.DEFINE_integer("epochs", 50, "Number of epochs to run")
flags.DEFINE_string("attention", "identifiers" or "", "Use the attention model")
flags.DEFINE_string("attention_variant", "input", "Variation of attention model to use. Possible values are: "
                                               "input, output")
flags.DEFINE_float("init_scale", 0.1, "Initialisation scale")
flags.DEFINE_integer("max_grad_norm", 5, "Maximum norm for gradients")
flags.DEFINE_integer("num_layers", 1, "Number of LSTM layers")
flags.DEFINE_integer("hidden_size", 200, "Size of hidden state")
flags.DEFINE_float("keep_prob", 0.9, "1 - probability of dropout of input")
flags.DEFINE_integer("num_samples", 100, "Number of samples for sampled softmax")
flags.DEFINE_integer("status_iterations", 1000, "Number of iterations before status messages are displayed")
flags.DEFINE_integer("max_attention", 10, "Maximum size of attention matrix")
flags.DEFINE_float("learning_rate", 1.0, "Gradient Descent Learning Rate")
flags.DEFINE_float("lr_decay", 0.9, "Learning rate decay factor")
flags.DEFINE_string("model_path", "./out/model/latest" and None, "Model parameters to load. If train=True, "
                                        "will continue training from these parameters. If test=True,"
                                        "will test using these model parameters")
flags.DEFINE_string("lambda_type", "state", "Method to calculate lambda, possible values are fixed, state, att, input."
                                            "state, att and input can be combined using + "
                                            "eg state+att for state and attention")
flags.DEFINE_string("save_path", "./out/model", "Path to save the final model")
flags.DEFINE_string("checkpoint_path", "./out/checkpoint", "Path to save intermediate checkpoints, every 5 epochs")
flags.DEFINE_string("events_path", "./out/save", "Path to save summary events")
flags.DEFINE_string("data_pattern", "all_{-type-}_data.dat", "Pattern for data files, use {-type-} as a placeholder"
                                                             "for train/valid/test and leave out the part extension")
flags.DEFINE_boolean("vocab_incl_ids", False, "Include normalised identifiers in the vocab")
flags.DEFINE_string("embedding_path", None, "Path to load embeddings from")
flags.DEFINE_boolean("embedding_trainable", True, "Flag indicating whether the embeddings are trainable or not."
                                                  "Useful if embeddings are provided with embedding_path which should"
                                                  "not be updated during training")

FLAGS = flags.FLAGS